### Задание 1   

### Обязательная часть
Будем парсить страницу со свежеми новостям на [habr.com](https://habr.com/ru/all//)
   
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
   
KEYWORDS = ['python', 'парсинг']
   
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
   
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>
   
### Дополнительная часть (необязательная)   
   
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
   
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
   
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [18]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [19]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all'

In [20]:
res = requests.get(URL) ### метод get
res

<Response [200]>

In [21]:
soup = BeautifulSoup(res.text, 'html.parser') ### при html параметр всегда 'html.parser' 
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<meta content="444736788986613" property="fb:app_id">
<meta content="website" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="Хабр" property="og:site_name"/>
<link href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" rel="image_src"/>
<meta content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="628" property="og:image:height"/>
<meta content="Все публикации подряд / Хабр" property="og:title"/>
<meta content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом." property="og:description"/>
<meta content="71593b225aeafc4e" name="yandex-verification"/>
<meta content="uns

In [22]:
news_blocks = soup.find_all('article', class_='post post_preview') ### извлекаем превью постов 
news_blocks

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/olegrok/" title="Автор публикации">
 <span class="default-image_mini default-image_pink">
 <svg aria-hidden="true" class="icon-svg" height="24" role="img" version="1.1" viewbox="0 0 24 24" width="24"><path d="M21.5 24h-19c-1.379 0-2.5-1.122-2.5-2.5v-19c0-1.379 1.122-2.5 2.5-2.5h19c1.379 0 2.5 1.122 2.5 2.5v19c0 1.379-1.122 2.5-2.5 2.5zm-19-23c-.827 0-1.5.673-1.5 1.5v19c0 .827.673 1.5 1.5 1.5h19c.827 0 1.5-.673 1.5-1.5v-19c0-.827-.673-1.5-1.5-1.5h-19zM15.598 12.385zM19.438 15.417l-.002-.005v-.001c-.875-2.226-2.484-3.054-3.445-3.549l-.273-.143c.029-.497-.025-1.034-.167-1.599l-.128.032.123-.044c-.765-2.152-1.757-2.585-2.632-2.967l-.006-.003-.535-2.121c.357-.065.628-.375.628-.752.001-.423-.342-.765-.765-.765s-.766.342-.766.765c0 .358.248.657.581.74l-.825 1.654-.014-.003-.024-.003c-1.053-.033-1.842.369-2.5.947-.633-.322-1.515-.729-2.158-1.814.107-

In [23]:
for news in news_blocks:
    news_lower = news.text.lower()
    if any([i in news_lower for i in KEYWORDS]): ### # ищем вхождение хотя бы одного желаемого хаба
        date = news.find('span', class_='post__time').text
        title = news.find('a', class_='post__title_link').text
        url = news.find('a', class_='post__title_link').get('href')
        frame = (pd.concat([pd.DataFrame([{'date': date,'title': title,'url': url}])], ignore_index=True))
frame

,date,title,url
0,сегодня в 13:58,Tarantool и кодогенерация на Lua,https://habr.com/ru/company/mailru/blog/557266/


In [24]:
df = pd.DataFrame() ### запишем данные в датафрейм
for page in range(50):
    time.sleep(0.25)   ### задержка исполнения
    all_soup = BeautifulSoup(requests.get(f'https://habr.com/ru/all/page{page+1}/').text, 'html.parser')
    for news in all_soup.find_all('article', class_='post post_preview'):
        if any([i in news.text.lower() for i in KEYWORDS]): 
            date = news.find('span', class_='post__time').text
            title = news.find('a', class_='post__title_link').text
            url = news.find('a', class_='post__title_link').get('href')
            df = (pd.concat([df,pd.DataFrame([{'date': date,'title': title,'url': url,'page': str(page+1)}])], ignore_index=True))
df

,date,title,url,page
0,сегодня в 13:58,Tarantool и кодогенерация на Lua,https://habr.com/ru/company/mailru/blog/557266/,1
1,сегодня в 09:07,«Маяки» в ЕГРЮЛ с python,https://habr.com/ru/post/558218/,2
2,вчера в 18:40,Топ 6 библиотек Python для визуализации: какую...,https://habr.com/ru/company/otus/blog/558478/,3
3,вчера в 16:52,OpenCV в Python: Часть 1 — Работа с изображени...,https://habr.com/ru/company/otus/blog/558426/,3
4,вчера в 16:50,Книга «Обработка естественного языка. Python и...,https://habr.com/ru/company/piter/blog/556140/,3
...,...,...,...,...
75,5 мая 2021 в 09:18,Структурированное сопоставление с шаблонами в ...,https://habr.com/ru/post/555804/,47
76,4 мая 2021 в 17:11,Звездные войны или подробный гайд по dplyr,https://habr.com/ru/post/555720/,48
77,4 мая 2021 в 16:30,PyTest для машинного обучения — простой учебни...,https://habr.com/ru/company/skillfactory/blog/...,49
78,4 мая 2021 в 14:45,Как я уместил систему управления товарами на с...,https://habr.com/ru/post/555696/,49


In [25]:
def get_hab_news(page):
    df = pd.DataFrame() ### запишем данные в датафрейм
    for page in range(50):
        time.sleep(0.25)   ### задержка исполнения
        all_soup = BeautifulSoup(requests.get(f'https://habr.com/ru/all/page{page+1}/').text, 'html.parser')
        for news in all_soup.find_all('article', class_='post post_preview'):
            if any([i in news.text.lower() for i in KEYWORDS]): 
                date = news.find('span', class_='post__time').text
                title = news.find('a', class_='post__title_link').text
                url = news.find('a', class_='post__title_link').get('href')
                df = (pd.concat([df,pd.DataFrame([{'date': date,'title': title,'url': url,'page': str(page+1)}])], ignore_index=True))
    return df

In [26]:
df = get_hab_news(page)
df

,date,title,url,page
0,сегодня в 13:58,Tarantool и кодогенерация на Lua,https://habr.com/ru/company/mailru/blog/557266/,1
1,сегодня в 09:07,«Маяки» в ЕГРЮЛ с python,https://habr.com/ru/post/558218/,2
2,вчера в 18:40,Топ 6 библиотек Python для визуализации: какую...,https://habr.com/ru/company/otus/blog/558478/,3
3,вчера в 16:52,OpenCV в Python: Часть 1 — Работа с изображени...,https://habr.com/ru/company/otus/blog/558426/,3
4,вчера в 16:50,Книга «Обработка естественного языка. Python и...,https://habr.com/ru/company/piter/blog/556140/,3
...,...,...,...,...
75,5 мая 2021 в 09:18,Структурированное сопоставление с шаблонами в ...,https://habr.com/ru/post/555804/,47
76,4 мая 2021 в 17:11,Звездные войны или подробный гайд по dplyr,https://habr.com/ru/post/555720/,48
77,4 мая 2021 в 16:30,PyTest для машинного обучения — простой учебни...,https://habr.com/ru/company/skillfactory/blog/...,49
78,4 мая 2021 в 14:45,Как я уместил систему управления товарами на с...,https://habr.com/ru/post/555696/,49


### Задание 2
   
### Обязательная часть
   
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
   
В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>
   
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.
   
### Дополнительная часть (необязательная)
   
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: [ссылка](https://vk.com/dev/methods) , вам поможет метод wall.get
   
   GROUP = 'netology'     
   TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ   
      
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json

In [9]:
emails = ['slim0825@gmail.com']

In [27]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {'emailAddresses': emails}
headers = {
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
          }
resp = requests.post(URL, data=json.dumps(params), headers=headers)
resp

<Response [200]>

In [28]:
data = json.loads(resp.text)
data

{'breaches': {'17609': {'breachId': 17609,
   'site': 'vk.com',
   'recordsCount': 42536910,
   'description': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and additional personal information. This breach is being privately shared on the internet.',
   'publishDate': '2020-05-21T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 0, 'emails': 20549631}},
  '16431': {'breachId': 16431,
   'site': 'houzz.com',
   'recordsCount': 53236067,
   'description': 'In July 2018, the housing site Houzz was allegedly breached, exposing information on over 55 million of its users.  The exposed information includes usernames, emails, password hashes, sign up times, last login times, location data, and ip addresses.  The data has been sold on at least one dark web market and is also being privately shared amongst several criminal groups.',
   'publishDate': '2018-12-13T00:00:00Z',
   'statistics': {'user

In [29]:
leak = []
for i in data['data']:
    for x in data['data'][i]:
        for breach_id in data['data'][i][x]:
            leak.append({
                        'email':x,
                        'breachId':breach_id['breachId']
                        })

In [30]:
mail_leak = pd.DataFrame(leak)
mail_leak

,email,breachId
0,slim0825@gmail.com,2917
1,slim0825@gmail.com,16401
2,slim0825@gmail.com,16431
3,slim0825@gmail.com,18177
4,slim0825@gmail.com,60
5,slim0825@gmail.com,17609
6,slim0825@gmail.com,32056


In [31]:
leak2 = []
for k,v in data['breaches'].items():
    leak2.append({
                'breachId':v['breachId'],
                'site':v['site'],
                'description':v['description'],
                'date':v['publishDate']
                })

descr_leak = pd.DataFrame(leak2)
descr_leak

,breachId,site,description,date
0,17609,vk.com,"At some time in 2020, the Russian social netwo...",2020-05-21T00:00:00Z
1,16431,houzz.com,"In July 2018, the housing site Houzz was alleg...",2018-12-13T00:00:00Z
2,60,qip.ru,"In 2011, Russian instant messaging service pro...",2016-11-01T00:00:00Z
3,16401,eyeem.com,"In February of 2018, the photography marketpla...",2018-12-13T00:00:00Z
4,18177,chelyabinsk-gid.info,"In November 2020, a collection of over 23,000 ...",2020-12-03T00:00:00Z
5,2917,mistertao.com,"In January 2017, MisterTao.com's user database...",2017-01-24T00:00:00Z
6,32056,megapolis-99.ru,"In November 2020, a collection of over 23,000 ...",2020-12-10T00:00:00Z


In [35]:
final_table = mail_leak.merge(descr_leak, on = 'breachId', how = 'left').filter(items = ['email', 'site', 'date', 'description'])
final_table

,email,site,date,description
0,slim0825@gmail.com,mistertao.com,2017-01-24T00:00:00Z,"In January 2017, MisterTao.com's user database..."
1,slim0825@gmail.com,eyeem.com,2018-12-13T00:00:00Z,"In February of 2018, the photography marketpla..."
2,slim0825@gmail.com,houzz.com,2018-12-13T00:00:00Z,"In July 2018, the housing site Houzz was alleg..."
3,slim0825@gmail.com,chelyabinsk-gid.info,2020-12-03T00:00:00Z,"In November 2020, a collection of over 23,000 ..."
4,slim0825@gmail.com,qip.ru,2016-11-01T00:00:00Z,"In 2011, Russian instant messaging service pro..."
5,slim0825@gmail.com,vk.com,2020-05-21T00:00:00Z,"At some time in 2020, the Russian social netwo..."
6,slim0825@gmail.com,megapolis-99.ru,2020-12-10T00:00:00Z,"In November 2020, a collection of over 23,000 ..."
